In [1]:
# Imports
import numpy as np
import pandas as pd
from time import time

from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
import keras.layers as layers
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.models import load_model

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     # Restrict TensorFlow to only use the first GPU
#     try:
#         tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
#         logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
#     except RuntimeError as e:
#         # Visible devices must be set before GPUs have been initialized
#         print(e)
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
              tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
# settings
BATCH_SIZE = 16
TOL = 0.0001
N_ITER_NO_CHANGE = 30
VERBOSE = 1
MAX_EPOCHS = 200
VALIDATION_FRACTION = 0.2

In [4]:
# loading train and test, creating validation
data1 = pd.read_csv('E:/Python/data/digits/train.csv')
data2 = pd.read_csv('E:/Python/data/digits/test.csv')
train = {}
test = {}
train['features'] = data1.iloc[:,1:]
train['labels'] = data1.iloc[:,:1]
train['features'] = np.expand_dims(train['features'], axis=(1, 0))
train['features'] = np.reshape(train['features'], (-1, 28, 28, 1))
test['features'] = data2
test['features'] = np.expand_dims(test['features'], axis=(1, 0))
test['features'] = np.reshape(test['features'], (-1, 28, 28, 1))
print("Image Shape: {}".format(train['features'][0].shape))
validation = {}
train['features'], validation['features'], train['labels'], validation['labels'] = \
    train_test_split(train['features'], train['labels'], test_size=VALIDATION_FRACTION, random_state=0)

Image Shape: (28, 28, 1)


In [5]:
# Pad images with 0s
train['features']      = np.pad(train['features'], ((0,0),(2,2),(2,2),(0,0)), 'constant')
validation['features'] = np.pad(validation['features'], ((0,0),(2,2),(2,2),(0,0)), 'constant')
test['features']       = np.pad(test['features'], ((0,0),(2,2),(2,2),(0,0)), 'constant')
    
print("Updated Image Shape: {}".format(train['features'][0].shape))

Updated Image Shape: (32, 32, 1)


In [1]:
# model itself
model = keras.Sequential()

model.add(layers.Conv2D(filters=6, kernel_size=(3, 3), input_shape=(32,32,1)))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.AveragePooling2D())

model.add(layers.Conv2D(filters=16, kernel_size=(3, 3)))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.AveragePooling2D())

model.add(layers.Flatten())

model.add(layers.BatchNormalization())

model.add(layers.Dense(units=120))
model.add(layers.BatchNormalization())
model.add(layers.Activation('elu'))

model.add(layers.Dense(units=84))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.Dropout(0.33))

model.add(layers.Dense(units=30))
model.add(layers.BatchNormalization())
model.add(layers.Activation('elu'))

model.add(layers.Dropout(0.33))

model.add(layers.Dense(units=10, activation = 'softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(lr=0.001, \
    beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), metrics=['accuracy'])

# model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.SGD(), metrics=['accuracy'])

# simple early stopping
es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=TOL, verbose=VERBOSE, patience=N_ITER_NO_CHANGE)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=VERBOSE, save_best_only=True)

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))


NameError: name 'keras' is not defined

In [24]:
# fit model and save the best
history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=MAX_EPOCHS, 
                    validation_data=validation_generator, validation_steps=validation_steps, 
                    shuffle=True, callbacks=[tensorboard, es, mc])
saved_model = load_model('best_model.h5')

Epoch 1/200
1047/1050 [============================>.] - ETA: 0s - loss: 0.3758 - accuracy: 0.8985 ETA: 0s - loss: 0
Epoch 00001: val_accuracy improved from -inf to 0.97209, saving model to best_model.h5
1050/1050 [==============================] - 20s 19ms/step - loss: 0.3751 - accuracy: 0.8986 - val_loss: 0.0909 - val_accuracy: 0.9721
Epoch 2/200
1049/1050 [============================>.] - ETA: 0s - loss: 0.1491 - accuracy: 0.9573
Epoch 00002: val_accuracy improved from 0.97209 to 0.97746, saving model to best_model.h5
1050/1050 [==============================] - 15s 15ms/step - loss: 0.1491 - accuracy: 0.9572 - val_loss: 0.0697 - val_accuracy: 0.9775
Epoch 3/200
1050/1050 [==============================] - ETA: 0s - loss: 0.1173 - accuracy: 0.9658
Epoch 00003: val_accuracy improved from 0.97746 to 0.97865, saving model to best_model.h5
1050/1050 [==============================] - 16s 15ms/step - loss: 0.1173 - accuracy: 0.9658 - val_loss: 0.0721 - val_accuracy: 0.9786
Epoch 4/200
1

KeyboardInterrupt: 

In [4]:
# loading train and test, creating validation
data1 = pd.read_csv('E:/Python/data/digits/train.csv')
data2 = pd.read_csv('E:/Python/data/digits/test.csv')
train = {}
test = {}
train['features'] = data1.iloc[:,1:]
train['labels'] = data1.iloc[:,:1]
train['features'] = np.expand_dims(train['features'], axis=(1, 0))
train['features'] = np.reshape(train['features'], (-1, 28, 28, 1))
test['features'] = data2
test['features'] = np.expand_dims(test['features'], axis=(1, 0))
test['features'] = np.reshape(test['features'], (-1, 28, 28, 1))
print("Image Shape: {}".format(train['features'][0].shape))
validation = {}

# Pad images with 0s
train['features']      = np.pad(train['features'], ((0,0),(2,2),(2,2),(0,0)), 'constant')
test['features']       = np.pad(test['features'], ((0,0),(2,2),(2,2),(0,0)), 'constant')

Image Shape: (28, 28, 1)


In [9]:
from sklearn.model_selection import train_test_split, KFold
import scipy
first_col = True
cross_fold = KFold(n_splits = 5, shuffle=True)
for train_index, test_index in cross_fold.split(train['features']):
    train_f = data1.iloc[:,1:]
    train_l = data1.iloc[:,:1]
    train_f = np.expand_dims(train_f, axis=(1, 0))
    train_f = np.reshape(train_f, (-1, 28, 28, 1))
    # Pad images with 0s
    train_f      = np.pad(train_f, ((0,0),(2,2),(2,2),(0,0)), 'constant')
    
    validation_f, validation_l = train_f[test_index], train_l.iloc[test_index]
    train_f, train_l = train_f[train_index], train_l.iloc[train_index]
    X_train, y_train = train_f, to_categorical(train_l)
    X_validation, y_validation = validation_f, to_categorical(validation_l)

    train_generator = ImageDataGenerator().flow(X_train, y_train, batch_size=BATCH_SIZE)
    validation_generator = ImageDataGenerator().flow(X_validation, y_validation, batch_size=BATCH_SIZE)
    
    # model itself
    model = keras.Sequential()

    model.add(layers.Conv2D(filters=6, kernel_size=(3, 3), input_shape=(32,32,1)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))
    model.add(layers.AveragePooling2D())

    model.add(layers.Conv2D(filters=16, kernel_size=(3, 3)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))
    model.add(layers.AveragePooling2D())

    model.add(layers.Flatten())

    model.add(layers.BatchNormalization())

    model.add(layers.Dense(units=120))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))

    model.add(layers.Dense(units=84))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(units=30))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(units=10, activation = 'softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(lr=0.002, \
        beta_1=0.9, beta_2=0.999, epsilon=1e-08), metrics=['accuracy'])

    # model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.SGD(), metrics=['accuracy'])

    # simple early stopping
    es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=TOL, verbose=VERBOSE, patience=N_ITER_NO_CHANGE)
    mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=VERBOSE, save_best_only=True)

    tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
    
    rlrop = ReduceLROnPlateau(monitor='accuracy', factor=0.5, patience=5, verbose=1)

    # fit model
    steps_per_epoch = X_train.shape[0]//BATCH_SIZE
    validation_steps = X_validation.shape[0]//BATCH_SIZE
    
    # fit model and save the best
    history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=MAX_EPOCHS, 
                        validation_data=validation_generator, validation_steps=validation_steps, 
                        shuffle=True, callbacks=[tensorboard, es, mc, rlrop])
    saved_model = load_model('best_model.h5')
    
    probs = saved_model.predict(test['features'])
#     cbr.fit(X=training_X_St.iloc[train_index], y=training_y[train_index], 
#         eval_set=(training_X_St.iloc[test_index], training_y[test_index]), use_best_model=True)
    predict = probs.argmax(axis=1)
    if first_col:
        pr_values = np.array(predict, ndmin=2)
        pr_values = np.transpose(pr_values)
        first_col = False
    else:
        pr_values = np.insert(pr_values, -1, predict, axis=1)
pr_values= scipy.stats.mode(pr_values, axis=1)

Epoch 1/200
2099/2100 [============================>.] - ETA: 0s - loss: 0.4855 - accuracy: 0.8572
Epoch 00001: val_accuracy improved from -inf to 0.97143, saving model to best_model.h5
2100/2100 [==============================] - 31s 15ms/step - loss: 0.4854 - accuracy: 0.8572 - val_loss: 0.0934 - val_accuracy: 0.9714
Epoch 2/200
2098/2100 [============================>.] - ETA: 0s - loss: 0.2410 - accuracy: 0.9336
Epoch 00002: val_accuracy improved from 0.97143 to 0.97940, saving model to best_model.h5
2100/2100 [==============================] - 26s 12ms/step - loss: 0.2408 - accuracy: 0.9337 - val_loss: 0.0687 - val_accuracy: 0.9794
Epoch 3/200
2100/2100 [==============================] - ETA: 0s - loss: 0.1902 - accuracy: 0.9473
Epoch 00003: val_accuracy did not improve from 0.97940
2100/2100 [==============================] - 27s 13ms/step - loss: 0.1902 - accuracy: 0.9473 - val_loss: 0.0804 - val_accuracy: 0.9754
Epoch 4/200
2097/2100 [============================>.] - ETA: 0s -

2099/2100 [============================>.] - ETA: 0s - loss: 0.0256 - accuracy: 0.9925
Epoch 00054: val_accuracy improved from 0.99333 to 0.99357, saving model to best_model.h5
2100/2100 [==============================] - 25s 12ms/step - loss: 0.0256 - accuracy: 0.9925 - val_loss: 0.0332 - val_accuracy: 0.9936
Epoch 55/200
2099/2100 [============================>.] - ETA: 0s - loss: 0.0287 - accuracy: 0.9918
Epoch 00055: val_accuracy did not improve from 0.99357
2100/2100 [==============================] - 24s 12ms/step - loss: 0.0287 - accuracy: 0.9918 - val_loss: 0.0351 - val_accuracy: 0.9931
Epoch 56/200
2096/2100 [============================>.] - ETA: 0s - loss: 0.0251 - accuracy: 0.9932
Epoch 00056: val_accuracy did not improve from 0.99357
2100/2100 [==============================] - 24s 12ms/step - loss: 0.0250 - accuracy: 0.9932 - val_loss: 0.0351 - val_accuracy: 0.9925
Epoch 57/200
2095/2100 [============================>.] - ETA: 0s - loss: 0.0253 - accuracy: 0.9930
Epoch 00

Epoch 108/200
2098/2100 [============================>.] - ETA: 0s - loss: 0.0143 - accuracy: 0.9955
Epoch 00108: val_accuracy did not improve from 0.99452
2100/2100 [==============================] - 24s 12ms/step - loss: 0.0142 - accuracy: 0.9955 - val_loss: 0.0326 - val_accuracy: 0.9943
Epoch 109/200
2099/2100 [============================>.] - ETA: 0s - loss: 0.0161 - accuracy: 0.9960
Epoch 00109: val_accuracy did not improve from 0.99452

Epoch 00109: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.
2100/2100 [==============================] - 25s 12ms/step - loss: 0.0161 - accuracy: 0.9960 - val_loss: 0.0329 - val_accuracy: 0.9942
Epoch 110/200
2096/2100 [============================>.] - ETA: 0s - loss: 0.0130 - accuracy: 0.9960
Epoch 00110: val_accuracy did not improve from 0.99452
2100/2100 [==============================] - 25s 12ms/step - loss: 0.0130 - accuracy: 0.9960 - val_loss: 0.0325 - val_accuracy: 0.9942
Epoch 111/200
2100/2100 [=====================

2100/2100 [==============================] - ETA: 0s - loss: 0.0501 - accuracy: 0.9865
Epoch 00040: val_accuracy did not improve from 0.99238
2100/2100 [==============================] - 24s 11ms/step - loss: 0.0501 - accuracy: 0.9865 - val_loss: 0.0378 - val_accuracy: 0.9918
Epoch 41/200
2095/2100 [============================>.] - ETA: 0s - loss: 0.0453 - accuracy: 0.9871
Epoch 00041: val_accuracy did not improve from 0.99238
2100/2100 [==============================] - 24s 11ms/step - loss: 0.0453 - accuracy: 0.9871 - val_loss: 0.0373 - val_accuracy: 0.9918
Epoch 42/200
2098/2100 [============================>.] - ETA: 0s - loss: 0.0482 - accuracy: 0.9864
Epoch 00042: val_accuracy did not improve from 0.99238
2100/2100 [==============================] - 24s 11ms/step - loss: 0.0483 - accuracy: 0.9864 - val_loss: 0.0392 - val_accuracy: 0.9917
Epoch 43/200
2096/2100 [============================>.] - ETA: 0s - loss: 0.0508 - accuracy: 0.9853
Epoch 00043: val_accuracy did not improve f

2098/2100 [============================>.] - ETA: 0s - loss: 0.0165 - accuracy: 0.9956
Epoch 00094: val_accuracy did not improve from 0.99286
2100/2100 [==============================] - 24s 11ms/step - loss: 0.0165 - accuracy: 0.9956 - val_loss: 0.0430 - val_accuracy: 0.9924
Epoch 95/200
2099/2100 [============================>.] - ETA: 0s - loss: 0.0156 - accuracy: 0.9958
Epoch 00095: val_accuracy did not improve from 0.99286
2100/2100 [==============================] - 24s 11ms/step - loss: 0.0156 - accuracy: 0.9958 - val_loss: 0.0426 - val_accuracy: 0.9925
Epoch 96/200
2100/2100 [==============================] - ETA: 0s - loss: 0.0158 - accuracy: 0.9958
Epoch 00096: val_accuracy did not improve from 0.99286
2100/2100 [==============================] - 24s 11ms/step - loss: 0.0158 - accuracy: 0.9958 - val_loss: 0.0421 - val_accuracy: 0.9927
Epoch 97/200
2096/2100 [============================>.] - ETA: 0s - loss: 0.0151 - accuracy: 0.9953
Epoch 00097: val_accuracy did not improve f

...

Epoch 17/200
2100/2100 [==============================] - ETA: 0s - loss: 0.0882 - accuracy: 0.9766
Epoch 00017: val_accuracy did not improve from 0.99226
2100/2100 [==============================] - 24s 11ms/step - loss: 0.0882 - accuracy: 0.9766 - val_loss: 0.0272 - val_accuracy: 0.9913
Epoch 18/200
2098/2100 [============================>.] - ETA: 0s - loss: 0.0802 - accuracy: 0.9778
Epoch 00018: val_accuracy did not improve from 0.99226
2100/2100 [==============================] - 24s 11ms/step - loss: 0.0801 - accuracy: 0.9779 - val_loss: 0.0364 - val_accuracy: 0.9899
Epoch 19/200
2098/2100 [============================>.] - ETA: 0s - loss: 0.0814 - accuracy: 0.9773
Epoch 00019: val_accuracy did not improve from 0.99226
2100/2100 [==============================] - 24s 11ms/step - loss: 0.0815 - accuracy: 0.9773 - val_loss: 0.0325 - val_accuracy: 0.9902
Epoch 20/200
2099/2100 [============================>.] - ETA: 0s - loss: 0.0746 - accuracy: 0.9796
Epoch 00020: val_accuracy did 

Epoch 45/200
2097/2100 [============================>.] - ETA: 0s - loss: 0.0462 - accuracy: 0.9865
Epoch 00045: val_accuracy improved from 0.99357 to 0.99417, saving model to best_model.h5
2100/2100 [==============================] - 24s 11ms/step - loss: 0.0462 - accuracy: 0.9865 - val_loss: 0.0263 - val_accuracy: 0.9942
Epoch 46/200
2095/2100 [============================>.] - ETA: 0s - loss: 0.0465 - accuracy: 0.9869
Epoch 00046: val_accuracy did not improve from 0.99417
2100/2100 [==============================] - 24s 11ms/step - loss: 0.0464 - accuracy: 0.9869 - val_loss: 0.0329 - val_accuracy: 0.9926
Epoch 47/200
2096/2100 [============================>.] - ETA: 0s - loss: 0.0446 - accuracy: 0.9874
Epoch 00047: val_accuracy did not improve from 0.99417
2100/2100 [==============================] - 24s 11ms/step - loss: 0.0446 - accuracy: 0.9874 - val_loss: 0.0292 - val_accuracy: 0.9924
Epoch 48/200
2100/2100 [==============================] - ETA: 0s - loss: 0.0449 - accuracy: 0.

Epoch 73/200
2095/2100 [============================>.] - ETA: 0s - loss: 0.0197 - accuracy: 0.9943
Epoch 00073: val_accuracy did not improve from 0.99417
2100/2100 [==============================] - 24s 11ms/step - loss: 0.0197 - accuracy: 0.9943 - val_loss: 0.0324 - val_accuracy: 0.9929
Epoch 74/200
2098/2100 [============================>.] - ETA: 0s - loss: 0.0206 - accuracy: 0.9942
Epoch 00074: val_accuracy did not improve from 0.99417
2100/2100 [==============================] - 24s 11ms/step - loss: 0.0206 - accuracy: 0.9942 - val_loss: 0.0330 - val_accuracy: 0.9926
Epoch 75/200
2098/2100 [============================>.] - ETA: 0s - loss: 0.0196 - accuracy: 0.9946
Epoch 00075: val_accuracy did not improve from 0.99417

Epoch 00075: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
2100/2100 [==============================] - 24s 11ms/step - loss: 0.0196 - accuracy: 0.9946 - val_loss: 0.0340 - val_accuracy: 0.9929
Epoch 00075: early stopping
Epoch 1/200
2095/2100

Epoch 51/200
2096/2100 [============================>.] - ETA: 0s - loss: 0.0316 - accuracy: 0.9906
Epoch 00051: val_accuracy did not improve from 0.99250
2100/2100 [==============================] - 24s 11ms/step - loss: 0.0316 - accuracy: 0.9906 - val_loss: 0.0326 - val_accuracy: 0.9911
Epoch 52/200
2098/2100 [============================>.] - ETA: 0s - loss: 0.0343 - accuracy: 0.9899
Epoch 00052: val_accuracy did not improve from 0.99250
2100/2100 [==============================] - 24s 12ms/step - loss: 0.0343 - accuracy: 0.9899 - val_loss: 0.0302 - val_accuracy: 0.9912
Epoch 53/200
2096/2100 [============================>.] - ETA: 0s - loss: 0.0330 - accuracy: 0.9909
Epoch 00053: val_accuracy did not improve from 0.99250

Epoch 00053: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
2100/2100 [==============================] - 24s 12ms/step - loss: 0.0329 - accuracy: 0.9910 - val_loss: 0.0377 - val_accuracy: 0.9913
Epoch 54/200
2098/2100 [=========================

Epoch 78/200
2097/2100 [============================>.] - ETA: 0s - loss: 0.0174 - accuracy: 0.9951
Epoch 00078: val_accuracy did not improve from 0.99321
2100/2100 [==============================] - 24s 12ms/step - loss: 0.0174 - accuracy: 0.9951 - val_loss: 0.0350 - val_accuracy: 0.9921
Epoch 79/200
2096/2100 [============================>.] - ETA: 0s - loss: 0.0186 - accuracy: 0.9951
Epoch 00079: val_accuracy did not improve from 0.99321
2100/2100 [==============================] - 24s 12ms/step - loss: 0.0185 - accuracy: 0.9951 - val_loss: 0.0358 - val_accuracy: 0.9921
Epoch 80/200
2095/2100 [============================>.] - ETA: 0s - loss: 0.0178 - accuracy: 0.9945
Epoch 00080: val_accuracy did not improve from 0.99321
2100/2100 [==============================] - 24s 12ms/step - loss: 0.0177 - accuracy: 0.9946 - val_loss: 0.0363 - val_accuracy: 0.9924
Epoch 81/200
2098/2100 [============================>.] - ETA: 0s - loss: 0.0163 - accuracy: 0.9947
Epoch 00081: val_accuracy did 

In [10]:
# predicting

data3 = pd.DataFrame(pr_values[0])
data3

,0
0,2
1,0
2,9
3,0
4,3
...,...
27995,9
27996,7
27997,3
27998,9


In [11]:
# saving results
data3.to_csv('E:/Python/data/digits/sample_submission24.csv')

In [52]:
pr_values

ModeResult(mode=array([[2],
       [0],
       [9],
       ...,
       [3],
       [9],
       [2]], dtype=int64), count=array([[5],
       [5],
       [5],
       ...,
       [5],
       [5],
       [5]]))